In [1]:
import os
import sys

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_dir = os.path.join(project_dir, 'src')
fig_dir = os.path.join(project_dir, 'fig')
data_dir = os.path.join(project_dir, 'data')
log_dir = os.path.join(project_dir, 'log')
os.makedirs(fig_dir, exist_ok=True)

# Add the src directory to sys.path
sys.path.append(src_dir)

import mech.full_DPSGD as DPSGDModule

In [2]:
data_args = {
    "method": "default",
    "data_dir": data_dir,
    "internal_result_path": "/scratch/bell/wei402/fdp-estimation/results"
}

args = DPSGDModule.generate_params(data_args=data_args, log_dir=log_dir, model_type="CNN")
sampler = DPSGDModule.DPSGDSampler(args)

Files already downloaded and verified


In [5]:
sample_P, sample_Q = sampler.preprocess(num_samples=200, num_workers=24)


Loading and projecting models with 24 workers:


Generating samples:   0%|          | 0/200 [00:00<?, ?it/s]/home/wei402/Desktop/fdp-estimation/src/mech/full_DPSGD.py:268: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stat

KeyboardInterrupt: 

In [4]:
print(sample_P,sample_Q)

[[-3.57414246]
 [-3.75797868]
 [-2.49597955]
 [-2.63993502]
 [-3.26186705]
 [-4.21204567]
 [-4.73753405]
 [-2.90314412]
 [-3.46055007]
 [-4.31558466]
 [-6.08034801]
 [-4.90389252]] [[-3.21461201]
 [-2.85231972]
 [-2.81492329]
 [-2.53532648]
 [-2.78180265]
 [-2.51536369]
 [-4.14006519]
 [-4.54838991]
 [-3.45309973]
 [-3.56675649]
 [-3.29761958]
 [-2.88039255]]
